In [1]:
using PauliPropagation

### X drive to ZZ target

In [11]:
#define hyperparams
nq = 4
del_t = 1.0
P= 20
tau = del_t * P # tau is total annealing time, divided into P steps/layers of circuit
topology = bricklayertopology(nq; periodic=false)
circuit = tfitrottercircuit(nq, P; topology=topology, start_with_ZZ=false) # 1st order trotter
#println(circuit)
nbonds = nq - 1 # if periodic nq bonds, else nq-1 (1d chain)
total = P * (nq + nbonds)
params  = Vector{Float64}(undef, total)

hbar = 1.0
G = -1.0
J = 1.0

idx = 1
for m in 1:P # layers of trotterized circuit
    #s_m = m / P
    s_m = (m - 0.5) / P #evaluate at middle of trotter slice

    gamma_m = s_m * del_t /hbar
    #println(gamma_m)   # paper Eq. (12)
    beta_m = (1-s_m) * del_t / hbar # paper Eq. (13a) - 1st order trotter
    #println(beta_m)
    theta_X = 2*beta_m * G 
    ##println(theta_X)              # angle for e^{-i theta_X X_j/2} - PauliRotation implementation in PP
    theta_ZZ = 2*gamma_m * J              # angle for e^{-i theta_ZZ Z_j Z_{j+1}/2}
    ##println(theta_ZZ)
    @inbounds fill!(view(params, idx:idx+nq-1), theta_X);             idx += nq # set X angles,then shift the index
    #println(params)
    @inbounds fill!(view(params, idx:idx+nbonds-1), theta_ZZ);       idx += nbonds
    #println(params)
end
H_ZZ = PauliSum(nq)
for pair in topology
    add!(H_ZZ, [:Z, :Z], collect(pair), J)
end

max_weight = 10
min_abs_coeff = 1e-7
propagated = propagate(circuit, H_ZZ, params)#; max_weight, min_abs_coeff)
println(overlapwithplus(propagated))


-2.9958176743671237


This works well, also when scaling up. Now, we want to do DQA for more complex Hamiltonians.

### X drive to TFIM target

In [12]:
#define hyperparams
using PauliPropagation  
nq = 4
del_t = 0.1
P= 500
tau = del_t * P # tau is total annealing time, divided into P steps/layers of circuit
topology = bricklayertopology(nq; periodic=false)
circuit = tfitrottercircuit(nq, P; topology=topology, start_with_ZZ=false) # 1st order trotter
#println(circuit)
nbonds = nq - 1 # if periodic nq bonds, else nq-1 (1d chain)
total = P * (nq + nbonds)
params  = Vector{Float64}(undef, total)

hbar = 1.0
G = 1.0
J = 1.0

idx = 1
for m in 1:P # layers of trotterized circuit
    # s_m = m / P
    s_m = (m - 0.5) / P

    gamma_m = J* s_m * del_t /hbar
    #println(gamma_m)   # paper Eq. (12)
    beta_m = G*del_t*(2*s_m-1)/ hbar #(1-s_m) * del_t / hbar # paper Eq. (13a) - 1st order trotter
    #println(beta_m)
    theta_X = 2*beta_m
    ##println(theta_X)              # angle for e^{-i theta_X X_j/2} - PauliRotation implementation in PP
    theta_ZZ = 2*gamma_m              # angle for e^{-i theta_ZZ Z_j Z_{j+1}/2}, J=1
    ##println(theta_ZZ)
    @inbounds fill!(view(params, idx:idx+nq-1), theta_X);             idx += nq # set X angles,then shift the index
    #println(params)
    @inbounds fill!(view(params, idx:idx+nbonds-1), theta_ZZ);       idx += nbonds
    #println(params)
end
H_tfim = PauliSum(nq)

for qind in 1:nq
    add!(H_tfim, :X, qind, G)
end

for pair in topology
    add!(H_tfim, [:Z, :Z], collect(pair), J)
end


max_weight = 10
min_abs_coeff = 1e-7
propagated = propagate(circuit, H_tfim, params)#; max_weight, min_abs_coeff)
println(overlapwithplus(propagated))

-4.670877390760635


This also works! We can make it more precise by increasing P, but we converge very slow.